In [7]:
! pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Librerías necesarias 
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\kracr\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\kracr\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\kracr\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\kracr\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\kracr\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

True

In [ ]:
# Importar el dataset
df = pd.read_csv('amazon.csv')
df

In [ ]:
# Cree una función para preprocesar el texto
def preprocess_text(text):

    # Tokenice el texto utilizando la función word_tokenize()
    # Pase el texto a minúscula

    # Elimine las stopwords utilizando stopwords.words('english')

    # Lematice los tokens utilizando WordNetLemmatizer()

    # Una los tokens de los procesos previos en una sola cadena

    # Retorne el texto procesado

# Aplique su función a la columna reviewText del dataframe

df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Obtenga su X y su y
X = ?
y = ?

# Codifique su y

# Particione el conjunto en entrenamiento, validación y prueba con train_test_split

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Hágale padding a sus secuencias y obtenga aquellas para entrenamiento y aquellas para prueba

max_words = ?
max_len = ?
tok = Tokenizer(num_words=?)
tok.fit_on_texts(?)
sequences_train = tok.texts_to_sequences(?)
sequences_val = tok.texts_to_sequences(?)
sequences_test = tok.texts_to_sequences(?)
sequences_matrix_train = pad_sequences(?,maxlen=?)
sequences_matrix_val = pad_sequences(?,maxlen=?)
sequences_matrix_test = pad_sequences(?,maxlen=?)


In [ ]:

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SimpleRNN

# Defina su modelo

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    # Incluya sus capas SimpleRNN s o LSTM, Dropout, etc
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
# Instancie su modelo

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=?,metrics=[?])

In [ ]:
# Entrene su modelo

history = model.fit(?,?,validation_data = (?,?),epochs = ?,batch_size=?,verbose = ?)

In [ ]:
# Evalúe su modelo

accr = model.evaluate(?,?)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
# Grafique sus resultados

import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history[?],label="Train");
plt.plot(history.history[?],label="Validation");
plt.title("Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show();